In [5]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import coco
import utils
import model as modellib
import visualize
from PIL import Image

#%matplotlib inline 

TargetGif = "Balls_2_NoContact.gif"

def extractFrames(inGif, outFolder):
    frame = Image.open(inGif)
    nframes = 0
    while frame:
        if (nframes<10):
            frame.save( '%s/%s-00%s.png' % (outFolder, os.path.basename(inGif), nframes ) , 'GIF')
        else:
            if (nframes<100):
                frame.save( '%s/%s-0%s.png' % (outFolder, os.path.basename(inGif), nframes ) , 'GIF')
            else:
                frame.save( '%s/%s-%s.png' % (outFolder, os.path.basename(inGif), nframes ) , 'GIF')
        nframes += 1
        try:
            frame.seek( nframes )
        except EOFError:
            break;
    return True

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# PART 1: take a gif and split it into frames.

OutputFolder = TargetGif[0:-4]
OutputPath = os.path.join(ROOT_DIR,OutputFolder)
# Create main output folder
if not os.path.exists(OutputPath):
    os.makedirs(OutputPath)
# Create folder where we'll store the raw frames
FramesFolder = os.path.join(OutputPath,"RawFrames")
if not os.path.exists(FramesFolder):
    os.makedirs(FramesFolder)
# Create folder where we'll store the raw frames
ProcessedFolder = os.path.join(OutputPath,"ObjectDetection")
if not os.path.exists(ProcessedFolder):
    os.makedirs(ProcessedFolder)

extractFrames(TargetGif, FramesFolder)

IMAGE_DIR = FramesFolder

class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(COCO_MODEL_PATH, by_name=True)
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

CSVOutput = open(OutputFolder+"/"+OutputFolder+"_DetectedObjects.csv","w") 
CSVOutput.write("frame,id,score,x,y,x2,y2\n")

file_names = next(os.walk(IMAGE_DIR))[2]
for i in range(len(file_names)):
    frame = file_names[i]
    image = skimage.io.imread(os.path.join(IMAGE_DIR, frame))
    results = model.detect([image], verbose=0)
    r = results[0]
    imagename = OutputFolder+'/ObjectDetection/frame_'+str(i)+'.png'
    visualize.save_instances(image, imagename, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])
    for PB in range(len(r['class_ids'])):
        CSVOutput.write(str(i)+","+str(class_names[r['class_ids'][PB]])+","+str(r['scores'][PB])+","+str(r['rois'][PB][0])+","+str(r['rois'][PB][1])+","+str(r['rois'][PB][2])+","+str(r['rois'][PB][3])+"\n")

CSVOutput.close() 



Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING     

array([173, 524, 221, 570], dtype=int32)